### Querying Movie-Rating_Reviewer Database

You've started a new movie-rating website, and you've been collecting data on reviewers' ratings of various movies. There's not much data yet, but you can still try out some interesting queries. Here's the schema:

Movie ( mID, title, year, director )
English: There is a movie with ID number mID, a title, a release year, and a director.

Reviewer ( rID, name )
English: The reviewer with ID number rID has a certain name.

Rating ( rID, mID, stars, ratingDate )
English: The reviewer rID gave the movie mID a number of stars rating (1-5) on a certain ratingDate.

Your queries will run over a small data set conforming to the schema. View the database. (You can also download the schema and data.)

Instructions: Each problem asks you to write a query in SQL.

In [1]:
# import libraries
import pandas as pd
import sqlite3

In [2]:
# connect to database
conn = sqlite3.connect(':memory:')

# create cursor object
cur = conn.cursor()

In [3]:
# read in csv files to dataframe and write records 
# stored in the DataFrame to a SQL database

df1 = pd.read_csv('movie.csv', )
df1.to_sql('movie', con = conn, index=False)
df2 = pd.read_csv('rating.csv', )
df2.to_sql('rating', con = conn, index=False)
df3 = pd.read_csv('reviewer.csv', )
df3.to_sql('reviewer', con = conn, index=False)

In [4]:
# view movie dataframe

cur.execute('''SELECT * FROM movie;''')            # sql query
movie = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
movie.columns = [x[0] for x in cur.description]    # labels dataframe columns
movie                                              # view dataframe

,mID,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [5]:
# view rating dataframe

cur.execute('''SELECT * FROM rating;''')            # sql query
rating = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
rating.columns = [x[0] for x in cur.description]    # labels dataframe columns
rating                                              # view dataframe

,rID,mID,stars,ratingDate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


In [6]:
# view reviewer dataframe

cur.execute('''SELECT * FROM reviewer;''')            # sql query
reviewer = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
reviewer.columns = [x[0] for x in cur.description]    # labels dataframe columns
reviewer                                              # view dataframe

,rID,name
0,201,Sarah Martinez
1,202,Daniel Lewis
2,203,Brittany Harris
3,204,Mike Anderson
4,205,Chris Jackson
5,206,Elizabeth Thomas
6,207,James Cameron
7,208,Ashley White


In [7]:
# 1. Find the titles of all movies directed by Steven Spielberg.

cur.execute("""SELECT title
               FROM movie
               WHERE director = 'Steven Spielberg'""")      # sql query
q1 = pd.DataFrame(cur.fetchall())                           # create dataframe from sql query
q1.columns = [x[0] for x in cur.description]                # labels dataframe columns
q1                                                          # view dataframe

,title
0,E.T.
1,Raiders of the Lost Ark


<img src="q1.png" />

In [8]:
# 2. Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.

cur.execute("""SELECT DISTINCT m.year           
               FROM movie m
               JOIN rating r
               ON m.mID = r.mID
               WHERE r.stars IN (4, 5)
               ORDER BY year""")                # sql query
q2 = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
q2.columns = [x[0] for x in cur.description]    # labels dataframe columns
q2                                              # view dataframe

,year
0,1937
1,1939
2,1981
3,2009


<img src="q2.png" />

In [9]:
# 3. Find the titles of all movies that have no ratings.

cur.execute("""SELECT DISTINCT m.title
               FROM movie m
               LEFT JOIN rating r
               ON m.mID = r.mID
               WHERE r.stars IS NULL""")         # sql query
q3 = pd.DataFrame(cur.fetchall())                       # create dataframe from sql query
q3.columns = [x[0] for x in cur.description]            # labels dataframe columns
q3                                                      # view dataframe

,title
0,Star Wars
1,Titanic


<img src="q3.png" />

In [10]:
# 4. Some reviewers didn't provide a date 
# with their rating. Find the names of 
# all reviewers who have ratings with a 
# NULL value for the date.

cur.execute("""SELECT rev.name
               FROM rating r
               JOIN reviewer rev
               ON r.rID = rev.rID
               WHERE r.ratingDate IS NULL""")   # sql query
q4 = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
q4.columns = [x[0] for x in cur.description]    # labels dataframe columns
q4                                              # view dataframe

,name
0,Daniel Lewis
1,Chris Jackson


<img src="q4.png" />

In [11]:
# 5. Write a query to return the ratings 
# data in a more readable format: 
# reviewer name, movie title, stars, 
# and ratingDate. Also, sort the data, 
# first by reviewer name, then by movie 
# title, and lastly by number of stars.

cur.execute("""SELECT rev.name, m.title, r.stars, r.ratingDate
               FROM movie m
               JOIN rating r
               ON m.mID = r.mID
               JOIN reviewer rev
               ON r.rID = rev.rID
               ORDER BY 1, 2, 3""")             # sql query
q5 = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
q5.columns = [x[0] for x in cur.description]    # labels dataframe columns
q5                                              # view dataframe

,name,title,stars,ratingDate
0,Ashley White,E.T.,3,2011-01-02
1,Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
2,Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
3,Brittany Harris,The Sound of Music,2,2011-01-20
4,Chris Jackson,E.T.,2,2011-01-22
5,Chris Jackson,Raiders of the Lost Ark,4,None
6,Chris Jackson,The Sound of Music,3,2011-01-27
7,Daniel Lewis,Snow White,4,None
8,Elizabeth Thomas,Avatar,3,2011-01-15
9,Elizabeth Thomas,Snow White,5,2011-01-19


<img src="q5.png" />

In [12]:
# 6. For all cases where the same reviewer 
# rated the same movie twice and gave it 
# a higher rating the second time, return the 
# reviewer's name and the title of the movie.

cur.execute("""SELECT rev.name, m.title
               FROM movie m
               JOIN rating r1 ON m.mID = r1.mID
               JOIN rating r2 ON r1.rID = r2.rID
               JOIN reviewer rev ON r2.rID = rev.rID
               WHERE r1.mID = r2.mID 
               AND r1.ratingDate > r2.ratingDate 
               AND r1.stars > r2.stars""")           # sql query
q6 = pd.DataFrame(cur.fetchall())                    # create dataframe from sql query
q6.columns = [x[0] for x in cur.description]         # labels dataframe columns
q6                                                   # view dataframe

,name,title
0,Sarah Martinez,Gone with the Wind


<img src="q6.png" />

In [13]:
# 7. For each movie that has at least one rating, 
# find the highest number of stars that movie 
# received. Return the movie title and number 
# of stars. Sort by movie title.

cur.execute("""SELECT m.title, MAX(r.stars) AS "Highest Rating"
               FROM movie m 
               JOIN rating r
               ON m.mID = r.mID
               WHERE r.stars >= 1
               GROUP BY m.title
               ORDER BY m.title""")             # sql query
q7 = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
q7.columns = [x[0] for x in cur.description]    # labels dataframe columns
q7                                              # view dataframe

,title,Highest Rating
0,Avatar,5
1,E.T.,3
2,Gone with the Wind,4
3,Raiders of the Lost Ark,4
4,Snow White,5
5,The Sound of Music,3


<img src="q7.png" />

In [14]:
# 8. For each movie, return the title and 
# the 'rating spread', that is, the 
# difference between highest and lowest 
# ratings given to that movie. Sort by 
# rating spread from highest to lowest, 
# then by movie title.

cur.execute("""SELECT m.title, MAX(r.stars) - MIN(r.stars) AS "Rating Spread"
               FROM movie m
               JOIN rating r
               ON m.mID = r.mID
               GROUP BY m.title
               ORDER BY 2 DESC, 1""")           # sql query
q8 = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
q8.columns = [x[0] for x in cur.description]    # labels dataframe columns
q8                                              # view dataframe

,title,Rating Spread
0,Avatar,2
1,Gone with the Wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
5,The Sound of Music,1


<img src="q8.png" />

In [15]:
# 9. Find the difference between the average 
# rating of movies released before 1980 
# and the average rating of movies released 
# after 1980. (Make sure to calculate the 
# average rating for each movie, then the 
# average of those averages for movies before 
# 1980 and movies after. Don't just calculate 
# the overall average rating before and after 1980.)

cur.execute("""SELECT AVG(before1980.avg) - AVG(after1980.avg) AS difference
FROM 
(SELECT AVG(r.stars) AS avg
FROM movie m
JOIN rating r
ON m.mID = r.mID
WHERE year < 1980
GROUP BY m.title) AS before1980,
(SELECT AVG(r.stars) AS avg
FROM movie m
JOIN rating r
ON m.mID = r.mID
WHERE year > 1980
GROUP BY m.title) AS after1980""")              # sql query
q9 = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
q9.columns = [x[0] for x in cur.description]    # labels dataframe columns
q9                                              # view dataframe

,difference
0,0.055556


<img src="q9.png" />